In [1]:
import json
from tqdm import tqdm
import numpy as np

In [2]:
train_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_train.txt'
dev_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_dev.txt'
test_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_test.txt'
train_original_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_train_original.txt'
all_data_in = '/mnt/scratch/chenqu/msmarco_wasi/original/all_data.txt'

train_out = '/mnt/scratch/chenqu/msmarco/original/session_train.txt'
dev_out = '/mnt/scratch/chenqu/msmarco/original/session_dev.txt'
test_out = '/mnt/scratch/chenqu/msmarco/original/session_test.txt'

In [3]:
target_query_set = set() # the last query in a session
all_sessions = []
with open(all_data_in) as fin:
    for line in tqdm(fin):
        session = json.loads(line.strip())
        all_sessions.append(session)
        
        target_query = session['query'][-1]['text']
        target_query_set.add(target_query)

276381it [04:04, 1130.71it/s]


In [6]:
len(target_query_set)

44971

In [7]:
np.random.seed(0)
dev_test_target_queries = np.random.choice(list(target_query_set), 2000, replace=False)
dev_target_queries = dev_test_target_queries[:1000]
test_target_queries = dev_test_target_queries[1000:]

In [8]:
dev_target_query_dict = {}
for query in dev_target_queries:
    dev_target_query_dict[query] = 1

test_target_query_dict = {}
for query in test_target_queries:
    test_target_query_dict[query] = 1

In [10]:
len(test_target_query_dict)

1000

In [11]:
train_sessions, dev_sessions, test_sessions = [], [], []
for session in tqdm(all_sessions):
    target_query = session['query'][-1]['text'] 
    if target_query in dev_target_query_dict:
        dev_sessions.append(session)
    elif target_query in test_target_query_dict:
        test_sessions.append(session)
    else:
        train_sessions.append(session)

100%|██████████| 276381/276381 [00:01<00:00, 148368.59it/s]


In [15]:
def write_sessions(sessions, out):
    with open(out, 'w') as fout:
        for session in tqdm(sessions):
            fout.write(json.dumps(session) + '\n')
            
write_sessions(dev_sessions, dev_out)
write_sessions(test_sessions, test_out)
write_sessions(train_sessions, train_out)

100%|██████████| 264257/264257 [02:11<00:00, 2008.40it/s]


In [14]:
dev_sessions[0]

{'session_id': '696j7zvzh6',
 'query': [{'id': '3c866x534qrr',
   'text': 'what are side effects associated with using pravastatin',
   'tokens': ['what',
    'are',
    'side',
    'effects',
    'associated',
    'with',
    'using',
    'pravastatin'],
   'type': 'DESCRIPTION',
   'candidates': [{'id': 'ipewiggk1xk51n2y',
     'url': 'https://www.webmd.com/drugs/2/drug-6989/pravastatin-oral/details/list-sideeffects',
     'url_tokens': ['www',
      'webmd',
      'com',
      'drugs',
      '2',
      'drug',
      '6989',
      'pravastatin',
      'oral',
      'details',
      'list',
      'sideeffects'],
     'title': 'Pravastatin SODIUM Side Effects by Likelihood and Severity',
     'content': 'Terms of Use ; About . Contact Us ; About WebMD ; Careers ; Newsletter ; Corporate ; WebMD Health Services ; Site Map ; Accessibility ; WebMD Network . Medscape ; Medscape Reference ; MedicineNet ; eMedicineHealth ; RxList ; OnHealth ; BootsWebMD ; WebMDRx ; First Aid ; WebMD Magazine 

In [ ]:
# inspect overlap:

In [3]:
train_query_dict = {}
with open(train_in) as fin:
    for line in fin:
        session = json.loads(line.strip())
        for turn_id, turn in enumerate(session['query']):
            query = turn['text']
            train_query_dict[query] = 1

In [14]:
def filter_dev(dev_in):
    lines_not_in_train = []
    with open(dev_in) as fin:
        for line in tqdm(fin):
            keep = True
            session = json.loads(line.strip())
            for turn_id, turn in enumerate(session['query']):
                query = turn['text']
                if query in train_query_dict:
                    keep = False
                    break
            if keep:
                lines_not_in_train.append(line)
    return lines_not_in_train

In [15]:
lines_not_in_train = filter_dev(all_data_in)

276381it [01:19, 3465.03it/s]


In [16]:
len(lines_not_in_train)

546